# Toronto anomalies analysis
Anomalies detection and performance measurement

### Importing necessary libraries

In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import utils
import os.path

### Loading the data

In [92]:
SITE = 7879
channels = pd.read_csv('../dataset/channels.csv')
channels[channels.siteId == SITE]

,siteId,channelId,channelName,units,isCalculated
81434,7879,11723,MP1 BTYVOLT,V,False
81435,7879,11722,MP1 UpDEPTH_1,mm,False
81436,7879,11721,MP1 UNIDEPTH,mm,False
81437,7879,11720,MP1 RAWVEL,m/s,False
81438,7879,11719,MP1 PEAKVEL_1,m/s,False
81439,7879,11718,MP1 PDEPTH_1,mm,False
81440,7879,14031,Raw Average Velocity,m/s,False
81441,7879,19696,MP1 VInterim,m/s,False
81442,7879,14232,Rainfall,mm,False
81443,7879,13428,MP1 SILT1,mm,False


In [96]:
D_FINAL = 11724
P_DEPTH_1 = 11718
Up_DEPTH_1 = 11722
D_INTERIM = 19294
MP1_BTYVOLT = 11723
Q_Raw = 14851
D_Raw = 14455
Daily_Cumulative_Rainfall = 22335
Data_Notes = 13828
MP1_QInterim = 19495
MP1_DInterim = 19294
V_Raw_Average = 14654
MP1_FLOW1 = 11717
MP1_VFINAL = 11725
MP1_QFINAL = 11726
Raw_Average_Velocity = 14031
Rainfall = 14232

In [97]:
dfinal = utils.load_channel_data(SITE, D_FINAL).rename('D_FINAL').fillna(0)
pdepth = utils.load_channel_data(SITE, P_DEPTH_1).rename('P_DEPTH_1').fillna(0)
updepth = utils.load_channel_data(SITE, Up_DEPTH_1).rename('Up_DEPTH_1').fillna(0)
dinterim = utils.load_channel_data(SITE, D_INTERIM).rename('D_Interim').fillna(0)

In [99]:
df = pd.concat([dfinal, pdepth, updepth, dinterim], axis=1).dropna()
df['pdepth_diff'] = np.abs(df.D_FINAL - df.P_DEPTH_1)
df['updepth_diff'] = np.abs(df.D_FINAL - df.Up_DEPTH_1)
df['dinterim_diff'] = np.abs(df.D_FINAL - df.D_Interim)
print('Anomalies in PDEPTH: {:}'.format(len(df[df['pdepth_diff'] > 1])))
print('Anomalies in UpDEPTH: {:}'.format(len(df[df['updepth_diff'] > 1])))
print('Anomalies in DInterim: {:}'.format(len(df[df['dinterim_diff'] > 1])))
print(df.head())
print(df.tail())
anomalies = df[df.updepth_diff > 1]
anomalies[['D_FINAL', 'Up_DEPTH_1', 'P_DEPTH_1']].head(20)

Anomalies in PDEPTH: 203920
Anomalies in UpDEPTH: 6152
Anomalies in DInterim: 0
                        D_FINAL   P_DEPTH_1  Up_DEPTH_1   D_Interim  \
time                                                                  
2015-10-06 12:55:00  486.119995  486.119995  485.589996  486.119995   
2015-10-06 13:00:00  502.399994  485.170013  502.399994  502.399994   
2015-10-06 13:05:00  498.489990  480.859985  498.489990  498.489990   
2015-10-06 13:10:00  499.910004  484.359985  499.910004  499.910004   
2015-10-06 13:15:00  498.869995  482.709991  498.869995  498.869995   

                     pdepth_diff  updepth_diff  dinterim_diff  
time                                                           
2015-10-06 12:55:00     0.000000      0.529999            0.0  
2015-10-06 13:00:00    17.229980      0.000000            0.0  
2015-10-06 13:05:00    17.630005      0.000000            0.0  
2015-10-06 13:10:00    15.550018      0.000000            0.0  
2015-10-06 13:15:00    16.160004      

,D_FINAL,Up_DEPTH_1,P_DEPTH_1
time,,,
2015-10-30 01:40:00,428.679993,184.360001,428.679993
2015-10-30 04:40:00,386.040009,209.600006,386.040009
2015-10-30 13:20:00,508.410004,15.880000,508.410004
2015-12-14 19:10:00,525.309998,614.890015,615.830017
2015-12-14 19:15:00,541.419983,614.419983,615.150024
2015-12-14 19:20:00,540.690002,651.119995,706.799988
2015-12-14 19:25:00,542.799988,591.450012,583.380005
2015-12-14 19:30:00,544.380005,590.700012,585.169983
2016-04-04 00:00:00,462.940002,18.230000,18.180000
